In [41]:
import pandas as pd
import json

file_path = 'dblp_v12.json'
lines_to_read = 100000

data_list = []  # List to hold data from lines

with open(file_path, 'r') as file:
    # Skip the first line
    for ind in range(9*lines_to_read+1):
        next(file)
    
    # Read specified number of lines
    for _ in range(lines_to_read):
        line = next(file)
        # print("===========")
        # print(line)
        if line != '}\n':  # Ensure it's not the last line (end of JSON object)
            if line[0]==",":
                paper_dict = json.loads(line[1:])
            else:
                paper_dict = json.loads(line)
            # print("**********")
            # print(line)
            # paper_dict = json.loads(line)
            # print("loaded")
            data_list.append(paper_dict)
        else:
            break  # Break loop if it's the last line

# Convert list of dictionaries to DataFrame
df = pd.DataFrame(data_list)

# Output DataFrame to CSV file
output_csv_path = 'output10.csv'
df.to_csv(output_csv_path, index=False)

# Print DataFrame (optional)
# print(df)

print(f"CSV file '{output_csv_path}' has been created.")


KeyboardInterrupt: 

In [ ]:
import pandas as pd

# List to hold individual DataFrames
dfs = []

# Loop through each CSV file
for i in range(1, 11):  # Assuming files are named 'output1.csv' to 'output10.csv'
    file_path = f'output{i}.csv'
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

output_csv_path = 'output_entire.csv'
combined_df.to_csv(output_csv_path, index=False)

print(f"CSV file '{output_csv_path}' has been created.")

In [ ]:
import pandas as pd

# List to hold individual DataFrames
features = ["id","authors","title","year","n_citation","doc_type","references","fos","venue"]

# Loop through each CSV file
file_path = f'output1.csv'
df = pd.read_csv(file_path)
df = df[features]
df = df.iloc[:5,:]
df

# paper node attributes

In [38]:
import pandas as pd
# file_path = f'output_entire.csv'
# df = pd.read_csv(file_path)
# features = ["id","authors","title","year","n_citation","doc_type","references","fos","venue"]
# df=df[features]
# print(df.shape)
# df.dropna(how='any', inplace=True)
# df.to_csv(output_csv_path, index=False)
# df.to_csv("output_entire_clean.csv", index=False)

file_path = f'output_entire_clean.csv'
df = pd.read_csv(file_path)
df_new = df[["id","title","year","n_citation","doc_type"]]
df_new["year"]=df_new["year"].astype(int)
df_new["n_citation"]=df_new["n_citation"].astype(int)
df_new["id"]="paper"+df_new["id"].astype(str)
df_new.head()
df_new.to_csv("papers.csv", index=False)

/var/folders/fs/r4jl6t2j2716kp94jc1zf4tc0000gn/T/ipykernel_79769/613467824.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["year"]=df_new["year"].astype(int)
/var/folders/fs/r4jl6t2j2716kp94jc1zf4tc0000gn/T/ipykernel_79769/613467824.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["n_citation"]=df_new["n_citation"].astype(int)
/var/folders/fs/r4jl6t2j2716kp94jc1zf4tc0000gn/T/ipykernel_79769/613467824.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

# paper-paper edge

In [54]:
file_path = f'output_entire_clean.csv'
df = pd.read_csv(file_path)

df_new = df[["id","references"]]
df_new = df_new.dropna().reset_index(drop = True)

import ast
df_new['references'] = df_new['references'].apply(ast.literal_eval)
df_new = df_new.explode("references").reset_index(drop = True)

df_new = df_new.explode("references")
df_new["id"] = "paper" + df_new["id"].astype(str)
df_new["references"] = "paper" + df_new["references"].astype(str)
print(df_new.shape)
df_new.to_csv("paper_paper.csv", index = False)

(7322280, 2)


# paper_author edge and author node attributes

In [41]:
file_path = f'output_entire_clean.csv'
df = pd.read_csv(file_path)

df_new = df[["id","authors"]]
import ast
df_new['authors'] = df_new['authors'].apply(ast.literal_eval)
df_new = df_new.explode("authors").reset_index(drop=True)

def extract_values(row):
    if "org" in row["authors"]:
        return pd.Series([row['authors']['name'], row['authors']['org'], row['authors']['id']])
    else:
        return pd.Series([row['authors']['name'], "", row['authors']['id']])
    

# Apply the function to create new columns
df_new[['author_name', 'author_org', 'author_id']] = df_new.apply(lambda row: extract_values(row), axis=1)

# Drop the original 'authors' column if needed
df_new.drop('authors', axis=1, inplace=True)
df_new["id"]="paper"+df_new["id"].astype(str)
df_new["author_id"]="author"+df_new["author_id"].astype(str)
print(df_new.shape)
df_new.head()

# save paper-author relationships
# df_ = df_new[["id","author_id"]]
# df_.to_csv("paper_author.csv", index = False)

# save author node attributes
df_ = df_new[["author_id","author_name","author_org"]]
df_=df_.drop_duplicates(subset=['author_id'])
print(df_.shape)
df_.to_csv("authors.csv", index = False)

/var/folders/fs/r4jl6t2j2716kp94jc1zf4tc0000gn/T/ipykernel_79769/1114433751.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['authors'] = df_new['authors'].apply(ast.literal_eval)


(2033988, 4)


,id,author_name,author_org,author_id
0,paper1091,Makoto Satoh,Shinshu University,author2312688602
1,paper1091,Ryo Muramatsu,Shinshu University,author2482909946
2,paper1091,Mizue Kayama,Shinshu University,author2128134587
3,paper1091,Kazunori Itoh,Shinshu University,author2101782692
4,paper1091,Masami Hashimoto,Shinshu University,author2114054191


# paper_venue edge and venue node attributes

In [86]:
file_path = f'output_entire_clean.csv'
df = pd.read_csv(file_path)
df_new = df[["id","venue"]]

import ast
df_new['venue'] = df_new['venue'].apply(ast.literal_eval)
# df_new = df_new.explode("authors")

def extract_values(row):
    # return pd.Series([row['venue']['raw'], row['venue']['id'], row['venue']['type']])
    if row["venue"]["raw"]=="":
        return pd.Series(["", "", ""])
    elif "id" not in row["venue"]:
        if "type" not in row["venue"]:
            return pd.Series([row['venue']['raw'], "", ""])
        else:
            return pd.Series([row['venue']['raw'], "", row['venue']['type']])
    else:
        return pd.Series([row['venue']['raw'], row['venue']['id'], row['venue']['type']])

# Apply the function to create new columns
df_new[['venue_name', 'venue_id', 'venue_type']] = df_new.apply(lambda row: extract_values(row), axis=1)

# # Drop the original 'authors' column if needed
df_new.drop('venue', axis=1, inplace=True)
df_new["id"]="paper"+df_new["id"].astype(str)
df_new["venue_id"]="venue"+df_new["venue_id"].astype(str)
print(df_new.head())

# paper_venue edges
df_ = df_new[["id","venue_id"]]
df_ = df_[df_.venue_id != "venue"]
print(df_.shape)
# df_ = df_new[["id","venue_id"]]
df_.to_csv("paper_venue.csv", index = False)


/var/folders/fs/r4jl6t2j2716kp94jc1zf4tc0000gn/T/ipykernel_79769/168251103.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['venue'] = df_new['venue'].apply(ast.literal_eval)
/var/folders/fs/r4jl6t2j2716kp94jc1zf4tc0000gn/T/ipykernel_79769/168251103.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[['venue_name', 'venue_id', 'venue_type']] = df_new.apply(lambda row: extract_values(row), axis=1)
/var/folders/fs/r4jl6t2j2716kp94jc1zf4tc0000gn/T/ipykernel_79769/168251103.py:22: SettingWithC

          id                                         venue_name  \
0  paper1091  International Conference on Human-Computer Int...   
1  paper1674        International Conference on Virtual Reality   
2  paper1688       Pattern Recognition and Machine Intelligence   
3  paper6522  International Symposium on Computer and Inform...   
4  paper8373  Asian Conference on Intelligent Information an...   

          venue_id venue_type  
0  venue1127419992          C  
1  venue2754954274          C  
2  venue1136274694          C  
3  venue1125967516          C  
4  venue1123338449          C  
(690779, 2)


In [92]:
df = df_new[["venue_id","venue_name","venue_type"]]
print(df.shape)
df=df[df.venue_id!="venue"]
print(df.shape)
df=df.drop_duplicates(subset=['venue_id'])
df.to_csv("venues.csv", index = False)

(691184, 3)
(690779, 3)


# paper_fos edge and fos node attributes

In [7]:
import pandas as pd
file_path = f'/home/ywang/GraphHT/datasets/DBLP-v5/output_entire_clean.csv'
df = pd.read_csv(file_path)

df_new = df[["id","fos"]]
# df_new=df_new.iloc[:10000,:]
print(df_new.shape)

import ast

df_new['fos'] = df_new['fos'].apply(ast.literal_eval)
df_new = df_new.explode("fos").reset_index(drop=True)

def extract_values(row):
    return pd.Series([row['fos']['name'], row['fos']['w']])


# Apply the function to create new columns
df_new[['fos_name', 'fos_w']] = df_new.apply(lambda row: extract_values(row), axis=1)

# Drop the original 'authors' column if needed
df_new.drop('fos', axis=1, inplace=True)
df_new["id"]="paper"+df_new["id"].astype(str)

df_new["fos_id"],_ = pd.factorize(df_new['fos_name'])
df_new["fos_id"]="fos"+df_new["fos_id"].astype(str)
df_ = df_new[["id","fos_id","fos_w"]]
df_.to_csv("paper_fos.csv", index = False)

df_ = df_new[["fos_id","fos_name"]]
df_ = df_.drop_duplicates(subset=['fos_id'])
df_.to_csv("fos.csv", index = False)

(10000, 2)


,id,fos_name,fos_w,fos_id
0,paper1091,Telecommunications network,0.45139,fos0
1,paper1091,Computer science,0.45245,fos1
2,paper1091,Mind map,0.53470,fos2
3,paper1091,Human–computer interaction,0.47011,fos3
4,paper1091,Multimedia,0.46629,fos4
